In [11]:
from preproses_signals.ECG import feature_ecg
from pathlib import Path

In [ ]:
import numpy as np
import pandas as pd
def read_data_haaglanden(
     root:str,
     
     number_persion : int = 1,
     print_data_analez : bool = False,
     stage_map = {
                    "Sleep stage W": 0, "W": 0,
                    "Sleep stage N1": 1, "Sleep stage 1": 1, "N1": 1,
                    "Sleep stage N2": 2, "Sleep stage 2": 2, "N2": 2,
                    "Sleep stage N3": 3, "Sleep stage 3": 3, "Sleep stage 4": 3, "N3": 3,
                    "Sleep stage R": 4, "R": 4, "REM": 4,
                    },
     win:int = 30
     ):

     psg_file = root / f"SN{number_persion:03d}.edf" # type: ignore

     raw = mne.io.read_raw_edf(psg_file, preload=True, stim_channel=None, verbose=False)

     data_x = raw.get_data()
     channel_name = raw.ch_names





     scoring_edf = root / f"SN{number_persion:03d}_sleepscoring.edf" # type: ignore

     # خواندن annotation ها
     ann = mne.read_annotations(scoring_edf)

     # تبدیل به DataFrame
     df = pd.DataFrame({
     "start_sec": ann.onset,
     "duration_sec": ann.duration,
     "label": ann.description
     })



     valid_labels = set(stage_map.keys())
     df_clean = df[df["label"].isin(valid_labels)].reset_index(drop=True)

     df_clean["label"] = df_clean["label"].map(stage_map)

     data_y = df_clean.astype(int).to_numpy()

     # save dataset 

     idxs, Xs, ys,n_p = [], [], [],[]
     list_start = list(df_clean['start_sec'])
     list_label = list(df_clean['label'])
     for i in range(len(list(df_clean['start_sec']))):
          start = list_start[i]
          label = list_label[i]
          end = start + win

          Xs.append(data_x[:, int(start):int(end)])               # یک پنجره به طول win
          # مثالِ ساده برای لیبل: میانگین/مود برچسب‌ها داخل پنجره
          y = label
          ys.append(y)
          idxs.append(start)                         # شروع پنجره (برای متادیتا)
          n_p.append(number_persion)
          X = np.stack(Xs, 0)                            # شکل: [N, C, win]
          y = np.array(ys) 
     if print_data_analez:
          print('-' * 50)
          print('x analize : ')
          print('shape x: ',data_x.shape) # type: ignore
          print('channel signal :',channel_name)
          print('-' * 50)
          print('y analize :')
          print('number vlaue :' ,df["label"].value_counts())
          print('number vlaue (clean label) :' ,df_clean["label"].value_counts())
          print('shape label:', data_y.shape)
          print('-' * 50)

     
     return data_x, data_y, X, y, np.array(idxs, dtype=np.int32),np.array(n_p, dtype=np.int32)
    

In [23]:
fs = 256
root = Path("/media/mohammad/NewVolume/signalDataset/haaglanden-medisch-centrum-sleep-staging-database-1.1/recordings")   # مسیر پوشه
import numpy as np
# X: [N, C, T]   y: [N]   meta: هرچه لازم داری (مثلاً subject_id, start_idx)
_,_, X, y, start_idx,subject_id = read_data_haaglanden(
     root = root,
     number_persion = 1,
     print_data_analez  = False,
     stage_map = {
                    "Sleep stage W": 0, "W": 0,
                    "Sleep stage N1": 1, "Sleep stage 1": 1, "N1": 1,
                    "Sleep stage N2": 2, "Sleep stage 2": 2, "N2": 2,
                    "Sleep stage N3": 3, "Sleep stage 3": 3, "Sleep stage 4": 3, "N3": 3,
                    "Sleep stage R": 4, "R": 4, "REM": 4,
                    },
     win = 30 * fs
     )
np.savez_compressed(
    "dataset.npz",
    X=X.astype("float32"),         # کم‌حجم‌تر از float64
    y=y.astype("int16"),
    subject_id=subject_id.astype("int16"),
    start_idx=start_idx.astype("int32")
)


/tmp/ipykernel_6516/1903896313.py:19: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_file, preload=True, stim_channel=None, verbose=False)
/tmp/ipykernel_6516/1903896313.py:19: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_file, preload=True, stim_channel=None, verbose=False)


KeyboardInterrupt: 